# 통계기반 자연어 처리 

## NLP의 접근 방식 
1. 규칙 기반 
    - 사람의 규칙(문법 / 패턴)을 정의
2. 통계 기반
    - 단어의 빈도, 확률, 통계를 활용
3. 딥러닝 기반 
    - 대규모 데이터 + 신경망 모델 활용

## N-gram 근사 
- Unigram 
    - N = 1 
    - 독립 단어 (하나의 단어)
- Bigram
    - N = 2
    - 바로 앞의 단어를 고려 
- Trigram
    - N = 3
    - 앞의 2개의 단어를 고려 

## 로그 확률
- 단어의 조건부 확률은 매우 작은 값 (0.0001, 0.00001)
- 조건부의 확률들 끼리 곱하게 되면 -> 0에 가까운 값 -> 계산이 불안정 
- 이러한 문제를 해결하기 위해 log 값을 사용 

## 혼란도(Preplexity)
- 문장을 얼마나 햇갈려하는지를 수치로 표현한 값
- 값이 높다면 -> 문장 이해도가 내려간다.
- 값이 낮다면 -> 문장 이해도가 올라간다. 

### BoW 
- 문서의 단어 순서는 무시 
- 각 문서에서 단어가 몇번 등장했는가?(빈도수)
- 가장 기본적인 백터화 방법 

In [ ]:
# 샘플 데이터셋 생성 
docs = [
    "영화가 정말 재미있었다", 
    "영화가 너무 지루하다", 
    "배우의 연기가 너무 좋았다."
]

In [ ]:
tokens = [doc.split() for doc in docs]
tokens

In [ ]:
# tokens에서 각각의 단어들을 하나의 리스트로 생성하고 중복은 제거 
# tokens의 2차원 리스트를 1차원으로 변환 
vocab1 = []
for token in tokens:
    # token -> tokens의 각 원소들 -> 1차원 리스트 
    for word in token:
        # word : token이라는 1차원 리스트의 각각의 원소들 
        vocab1.append(word)
vocab1 = list(set(vocab1))

In [ ]:
vocab1

In [ ]:
vocab = list(set(sum(tokens, [])))

In [ ]:
# Bow 행렬을 하나 생성 
# tokens의 문장에 vocab의 단어가 몇개 포함되어있는가?
bow_list = []
vocab.sort()
for doc in tokens:
    # row = [ doc.count(word) for word in vocab ]
    row = []
    for word in vocab:
        # count() 함수는 list에서 인자의 값과 같은 데이터가 몇개 있는가?
        row.append(doc.count(word))
    bow_list.append(row)

In [ ]:
import pandas as pd 

In [ ]:
df_bow = pd.DataFrame(bow_list, columns=vocab)

In [ ]:
df_bow

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(docs)

In [ ]:
df_vectorizer = pd.DataFrame(X.toarray(), 
    columns = vectorizer.get_feature_names_out())

In [ ]:
df_vectorizer

### TF-IDF 
- Bow 모델(단순 빈도 모델)의 한계를 보완하기 위한 통계 기반 기법

- TF
    - 문서(문장들의 집합) 내의 빈도가 높을수록 값이 큼
    - 자주 등장 할수록 그 문서에서 중요하다고 판단 
- IDF
    - 문장 내에서의 단어의 희소성

In [ ]:
import math 

In [ ]:
# 단어사전의 길이 
V = len(vocab)
# 전체 문서의 길이 
N = len(docs)

In [ ]:
tokens

In [ ]:
# 단어의 개수를 생성 
word_cnt = {
    w : sum(1 for doc in tokens if w in doc) for w in vocab
}
word_cnt

In [ ]:
# TF 계산식 함수 
def tf(word, doc):
    # word : 단어 사전의 각 원소
    # doc : tokens 각 원소 
    # doc.count(word) : 문장에서 특정 단어의 개수 
    # len(doc) : 문장의 단어의 개수
    result = doc.count(word) / len(doc)
    return result

In [ ]:
# IDF 계산식 함수 
def idf(word):
    # word : 단어 사전의 각 원소
    # N : docs의 길이 -> 문장들의 개수 
    # word_cnt[word] -> 문서에서 특정 단어의 개수
    result = math.log( (N) / (word_cnt[word] + 1) ) + 1
    return result

In [ ]:
# TF-IDF -> TF의 값과 IDF 값을 곱한 수치 
X_tfidf = [ 
    [tf(w, doc) * idf(w) for w in vocab] for doc in tokens 
]
X_tfidf

In [ ]:
pd.DataFrame(X_tfidf, columns = vocab)

In [ ]:
# TF-IDF는 scikit-learn에 class가 존재 
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vector = TfidfVectorizer(
    ngram_range= (1,1), 
    min_df=1
)
X =  vector.fit_transform(docs)
pd.DataFrame(
    X.toarray(), 
    columns = vector.get_feature_names_out()
)

- 감정 분석에서 TF-IDF + SVM의 조합이 보편적으로 사용
    - 단어별로 feature가 생성이 되기 때문에 고차원 데이터가 생성이 되고 value가 0이 데이터가 많은 비율을 차지 하기 때문에 트리 구조를 중복으로 사용하는 배깅 부스팅은 어울리지 않는다
- ngram_range는 (1,2)로 사용을 하게 되면 '너무 좋았다', '정말 별로다' 이러한 단어 패턴도 포착 가능 
- 고차원으로 생성이 된 행렬 데이터를 차원 축소 기법을 이용하여 차원의 수를 줄이고 모델에 학습하여 성능을 평가하는 방법도 존재 

### N-gram
- 단어를 몇 개까지 하나로 볼것인가?
- 언어의 모델을 문장의 자연스러움을 수치화하는 통계적 방법 
- 수치화 값으로는 로그확률, 혼란도 
- 로그확률은 매우 크지 않은 음수인 경우 자연스러움 표현 
- 혼란도 낮을수록 자연스러움을 표현  

In [ ]:
# 샘플 데이터를 생성 
texts = [
    '오늘 날씨가 좋다', 
    '오늘 기분이 좋다', 
    '내일 날씨가 맑다', 
    '기분이 아주 좋다', 
    '날씨가 아주 좋다'
]

In [ ]:
from konlpy.tag import Okt

In [ ]:
# 토큰화 함수 생성 
okt = Okt()
def tokenize(text):
    result = okt.morphs(text)
    return result

In [ ]:
# 문장의 시작과 끝 부분에 태그를 추가 
tokens = [ ['<s>'] + tokenize(text) + ['</s>'] for text in texts ]
tokens

In [ ]:
# unigram, bigram 카운트 확인 
from collections import Counter
unigram = Counter(w for token in tokens for w in token)
bigram = Counter(( token[i], token[i+1] ) for token in \
                 tokens for i in range(len(token) - 1) )
print(unigram)
print(bigram)

In [ ]:
# 위의 다중 for문을 풀어서 작성 
# for token in tokens:
#     for i in range(len(token) - 1):
#         print(token[i], token[i+1])

In [ ]:
# 문서 내에서 단어의 개수 
V = len(unigram)
print(V)

In [ ]:
# 확률이 0이 나오는 경우
# Add-1 Smoothing(Laplace Smoothing) 
# 분자에 + 1 -> 분모에는 + 문장에서의 단어의 개수  
def bigram_prob(prev, curr):
    # bigram[(prev, curr)] -> 연결된 두개의 단어의 개수
    # unigram[prev] -> 앞의 단어의 개수
    # 특정 단어 뒤에 단어가 나올 확률
    result = (bigram[(prev, curr)] + 1) / ( unigram[prev] + V)
    return result

# 로그 확률 -> 음수의 절대값이 크지 않다면 자연스러운 구조 
def log_prob(tokens):
    # tokens : 문장을 토큰화한 리스트
    # 문장의 시작과 끝에 태그 추가 
    seq = ['<s>'] + tokens + ['</s>']
    # 로그 확률의 누적합을 위해 초기 값 0.0 지정 
    log_p = 0.0
    for i in range(len(seq) - 1):
        # 확률 생성
        p = bigram_prob(seq[i], seq[i+1])
        # log_p에 누적합 
        log_p += math.log(p)
    return log_p


In [ ]:
# 혼란도 
def perplexity(tokens):
    # tokens : 문장을 토큰화 한 리스트
    log_p = log_prob(tokens)
    T = len(tokens) + 1
    # 자연상수 계산식
    result = math.exp(-log_p / T)
    return result

In [ ]:
# 학습된 unigram / bigram을 이용하여 확률을 계산하고 로그 확률, 
# 혼란도 확인 
new_texts = [
    '오늘 날씨가 맑다', 
    '기분이 내일 좋다', 
    '날씨가 아주 좋다'
]

result = []
for text in new_texts:
    tokens = tokenize(text)
    # 로그확률
    lp = log_prob(tokens)
    # 혼란도 
    perple = perplexity(tokens)
    result.append([text, lp, perple])


In [38]:
pd.DataFrame(result, columns = ['문장', '로그확률', '혼란도'])

,문장,로그확률,혼란도
0,오늘 날씨가 맑다,-8.536211,5.513735
1,기분이 내일 좋다,-9.694247,6.950749
2,날씨가 아주 좋다,-7.843064,4.799985


## 실습 문제 
- ratings_test.txt 파일을 로드 
- document 컬럼의 결측치를 제외하고 list 형태로 변환(데이터의 개수는 상위 5000개만)
- unigram, bigram 단어의 개수를 생성 
- 로그 확률, 혼란도 함수를 생성
- new_texts를 이용하여 로그확률, 혼란도를 생성 

In [ ]:
new_texts = [
    '이 영화 좋네', 
    '시간이 가는줄 몰랐다', 
    '아 이런 내 시간', 
    '연기가 그닥 ... 배우 ... '
]